In [5]:
import sys
import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd
import numpy as np
import requests.auth
from google.colab import files
import csv

In [3]:
# Input
ASIN = 'B09CGB6VRR' # The ASIN of the product
reviews = 100 # the number of comments to parse

In [6]:
title = []
stars = []
date = []
URLs = []
review = []

In [4]:
class CheckConnection:

    def __init__(self):
        pass

    # connection data
    def coonection_data(self):
        global data, HEADERS, proxyDict
        with open('proxy_data.csv', 'r') as csv_file:
            global csv_r
            csv_r = csv.reader(csv_file)
            data = [row for row in csv_r]
        proxyDict = {
            "https": data[0][0],
        }
        HEADERS = ({
            'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
            'Accept-Language': 'en-US, en;q=0.5'})

    # check proxy
    def check_proxy_data(self):
        self.coonection_data()
        try:
            url = 'https://amazon.com/'
            global r
            r = requests.get(url, headers=HEADERS, proxies=proxyDict)
            r = r.status_code
        except:
            print('Error in proxy data. Change proxy')
            self.delete_one_proxy()
            self.check_proxy_data

    # check amazon connection
    def check(self):
        self.check_proxy_data()
        if r == 200:
            return print('Connection with Amazon is established')
        else:
            self.delete_one_proxy()

    def delete_one_proxy(self):
        with open('proxy_data.csv', 'w', newline='') as csv_file:
            csv_w = csv.writer(csv_file)
            csv_w.writerows(data[1:])
        self.check()

In [7]:
class Parser:
    def __init__(self, asin, reviews, check_connection):
        self.asin = asin
        self.reviews = reviews
        self.check_connection = check_connection
        self.URLs = []
        self.title = []
        self.stars = []
        self.date = []
        self.review = []

    def check_asin(self):
        if len(self.asin) == 10:
            print('ASIN is correct')
        else:
            print('Wrong ASIN. Please check it')
            sys.exit(1)

    def reviews_num(self):
        if self.reviews >= 100:
            self.generate_urls()
        else:
            print('Too few reviews. Please, choose another ASIN')
            sys.exit(1)

    def generate_urls(self):
        num_pages = int((self.reviews + 10) / 10)  # Each page has up to 10 reviews
        for i in range(1, num_pages + 1):
            URL = (f'https://www.amazon.com/dp/product-reviews/{self.asin}/ref=cm_cr_arp_d_paging_btm_next_3?'
                   f'ie=UTF8&reviewerType=all_reviews&pageNumber={i}&sortBy=recent')
            self.URLs.append(URL)
        print(f"Generated {len(self.URLs)} URLs for scraping.")

    def get_data(self, proxyDict):
        HEADERS = self.check_connection.HEADERS
        for idx, url in enumerate(self.URLs, 1):
            try:
                print(f"Scraping URL {idx}: {url}")
                response = requests.get(url, headers=HEADERS, proxies=proxyDict, timeout=10)
                response.raise_for_status()
                soup = BeautifulSoup(response.text, "html.parser")

                # Extract titles
                title_new = soup.find_all(class_="a-size-base a-link-normal review-title a-color-base review-title-content a-text-bold")
                for t in title_new:
                    self.title.append(t.find('span').get_text(strip=True) if t.find('span') else "error")

                # Extract stars
                stars_new = soup.find_all('i', attrs={'data-hook':'review-star-rating'})
                for s in stars_new:
                    self.stars.append(s.find('span').get_text(strip=True) if s.find('span') else "error")

                # Extract dates
                date_new = soup.find_all(class_="a-size-base a-color-secondary review-date", attrs={'data-hook':'review-date'})
                for d in date_new:
                    self.date.append(d.get_text(strip=True) if d else "error")

                # Extract reviews
                review_new = soup.find_all(class_="a-size-base review-text review-text-content")
                for r in review_new:
                    self.review.append(r.find('span').get_text(strip=True) if r.find('span') else "error")

            except Exception as e:
                print(f"Error scraping {url}: {e}")
                continue  # Skip to the next URL

        # Verify data consistency
        if len(self.stars) == len(self.date) == len(self.review) == len(self.title):
            print("Data collection successful.")
        else:
            print("Data collection error: Mismatched lengths.")
            sys.exit(1)

In [12]:
def main():
    # Initialize CheckConnection
    cc = CheckConnection()
    cc.check()

    # Initialize Parser
    parser = Parser(ASIN, reviews, cc)
    parser.check_asin()
    parser.reviews_num()
    parser.get_data(cc.proxyDict)

In [ ]:
# Launch the scraper
if __name__ == "__main__":
    main()

In [ ]:
# Сonverting information into a table
df = pd.Series (stars, name = 'Stars').to_frame()
df ['Date'] = pd.Series (date, name = 'date').to_frame()
df ['Titles'] = pd.Series (title, name = 'Titles').to_frame()
df ['Reviews'] = pd.Series (review, name = 'Reviews').to_frame()
df ['Reviews'] = df ['Reviews'].replace ('\n  ','', regex=True).replace ('\n','', regex=True)
df.head ()

,Stars,Date,Titles,Reviews
0,1.0 out of 5 stars,"Reviewed in the United States on February 5, 2022",Broke after 6 weeks! Band split right in half.,My young adult child got these as a Christmas ...
1,5.0 out of 5 stars,"Reviewed in the United States on February 4, 2022",Sony noise cancelling wireless headphones,The sound is very good and the noise cancellin...
2,5.0 out of 5 stars,"Reviewed in the United States on February 4, 2022",Best headphones!,The noise cancellation is amazing!! I wasnt ex...
3,5.0 out of 5 stars,"Reviewed in the United States on February 4, 2022",Life changing - these are great headphones esp...,"I don't do a lot of reviews. However, reading ..."
4,2.0 out of 5 stars,"Reviewed in the United States on February 3, 2022",average sound,"Not really the greatest sound, the bass is not..."


In [ ]:
# download the data in excel file
df.to_excel('Reviews.xlsx', index=False)
files.download ('Reviews.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>